In [8]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

URL = "https://github.com/google/?tab=repositories"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ja,en-US;q=0.9,en;q=0.8"
}

response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

repo_items = soup.select(
    "li.col-12.d-flex.width-full.py-4.border-bottom.color-border-muted.public.source")

results = []

for item in repo_items:
    # 名前
    name_tag = item.select_one("a[itemprop='name codeRepository']")
    repo_name = name_tag.text.strip() if name_tag else "N/A"

    # 言語
    lang_tag = item.select_one("[itemprop='programmingLanguage']")
    language = lang_tag.text.strip() if lang_tag else "N/A"

    # スター
    star_tag = item.select_one("a[href*='/stargazers']")
    stars = star_tag.text.strip() if star_tag else "0"

    results.append((repo_name, language, stars))

    time.sleep(1)

# -----------------------------
# DB保存
conn = sqlite3.connect("repos.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS repositories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    language TEXT,
    stars INTEGER
)
""")

cur.executemany(
    "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)",
    results
)

conn.commit()

cur.execute("SELECT * FROM repositories")
rows = cur.fetchall()

print("\n=== 保存されたデータ ===")
for row in rows:
    print(row)

conn.close()


=== 保存されたデータ ===
